In [1]:
import gpytorch
# create a simple 1 d regression dataset for GP Regresssion
import numpy as np
import torch
import matplotlib.pyplot as plt
import gpytorch
import scipy.io as sio
from sklearn.model_selection import train_test_split

x = np.linspace(0, 1, 100)
y = np.sin(x * (2 * np.pi)) + np.random.randn(100) * 0.2
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=False)
x_train = torch.from_numpy(x_train).float().squeeze()
y_train = torch.from_numpy(y_train).float().squeeze()
x_test = torch.from_numpy(x_test).float().squeeze()
y_test = torch.from_numpy(y_test).float().squeeze()

#  create a gpytorch model 
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        # inner_kernel = (gpytorch.kernels.MaternKernel(nu=1.5) + gpytorch.kernels.RQKernel())*gpytorch.kernels.PeriodicKernel()
        # self.covar_module = gpytorch.kernels.ScaleKernel(inner_kernel)
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)
# train this using lbfgs optimizer
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train, y_train, likelihood)
model.train()
likelihood.train()
optimizer = torch.optim.LBFGS(model.parameters(), lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
def closure():
    optimizer.zero_grad()
    output = model(x_train)
    loss = -mll(output, y_train)
    return loss
losses = []
for i in range(100):
    loss = closure()
    losses.append(loss.item())
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, 100, loss.item()))
    optimizer.step(closure)
# test the model
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(x_test))
    mean = observed_pred.mean
    lower, upper = observed_pred.confidence_region()
# plot the results
f, ax = plt.subplots(1, 1, figsize=(4, 3))
ax.plot(x_train.numpy(), y_train.numpy(), 'k*')
ax.plot(x_test.numpy(), y_test.numpy(), 'b')
ax.plot(x_test.numpy(), mean.numpy(), 'r')
ax.fill_between(x_test.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)

# compare this result against adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
def closure():
    optimizer.zero_grad()
    output = model(x_train)
    loss = -mll(output, y_train)
    return loss
losses = []
for i in range(100):
    loss = closure()
    losses.append(loss.item())
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, 100, loss.item()))
    optimizer.step(closure)
# test the model
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    


ImportError: /home/gautam.pv/miniconda3/envs/akc_env_kkd/lib/python3.10/site-packages/torch/lib/libtorch_cuda_cpp.so: undefined symbol: cudaGraphRetainUserObject, version libcudart.so.11.0

In [2]:
#use periodic * RBF kernel and do not train on the periodic kernel and set its period to 2
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        inner_kernel = gpytorch.kernels.RBFKernel()
        self.covar_module = gpytorch.kernels.ScaleKernel(inner_kernel)
        self.covar_module.base_kernel.period_length.requires_grad = False
        self.covar_module.base_kernel.period_length = 2

    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)
# train this using lbfgs optimizer
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train, y_train, likelihood)
model.train()
likelihood.train()
optimizer = torch.optim.LBFGS(model.parameters(), lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
def closure():
    optimizer.zero_grad()
    output = model(x_train)
    loss = -mll(output, y_train)
    return loss
losses = []
for i in range(100):
    loss = closure()
    losses.append(loss.item())
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, 100, loss.item()))
    optimizer.step(closure)

In [ ]:
# use lbfgs to train a GP Regression model using GPYtorch
# initialize the model
# initialize the likelihood
likelihood = GaussianLikelihood()
model = ExactGPModel(torch.from_numpy(x).float().unsqueeze(-1), torch.from_numpy(y).float().unsqueeze(-1), likelihood)
# Find optimal model hyperparameters
# Use the adam optimizer
# "Loss" for GPs - the marginal log likelihood
# Defines the model and likelihood
model.train()
likelihood.train()
# Use the LBFGS optimizer
optimizer = torch.optim.LBFGS(model.parameters(), lr=0.1)
# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
# Define training iterations
training_iter = 100
for i in range(training_iter):
    # Zero backpropped gradients from previous iteration
    def closure():
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        return loss
    loss = optimizer.step(closure)
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iter, loss.item()))